# Monkeypatch

In the context of software testing, a monkeypatch is a technique used to dynamically modify or override functions or methods at runtime. The purpose of monkeypatching is often to replace or modify certain behaviors during testing without modifying the actual source code. It allows you to temporarily alter the behavior of functions or methods to suit the needs of your tests.

Here's a typical scenario where monkeypatch can be useful:

Let's say you have a function that relies on an external service, and you want to test this function in isolation without making actual network requests. Instead of modifying the function's code, you can use monkeypatch to temporarily replace the external service call with a mock or stub during testing.

In Python, the pytest testing framework provides a built-in monkeypatch fixture that you can use to modify attributes, override functions, or set environment variables during tests.

In [1]:
import pytest
import math_operation

def custom_add2(x, y):
    return x * y

def test_addition_with_mocked_function2():
    # Create a monkeypatch object
    monkeypatch = pytest.MonkeyPatch()

    # Use monkeypatch to set the 'add' function to the custom_add function
    monkeypatch.setattr(math_operation, 'add', custom_add2)

    # Call the 'add' function from the math_operation module, which is now modified
    result = math_operation.add(3, 4)

    # Print the result
    print(result)  # This will print 12

    # Undo the monkeypatch modification - Really important (If I wouldn't include it, then I couldn't revert it back later on.)
    monkeypatch.undo()

test_addition_with_mocked_function2()

print(math_operation.add(3,4)) # Output: 7 (Because of monkeypatch.undo())

12
7


In case you want to run pytest and monkeypatch as a standalone file, you can do it like this:

In [ ]:
# monkeypatch.py

import math_operation
import pytest

def custom_add(x, y):
    return x * y

@pytest.fixture
def monkeypatched_add(monkeypatch):
    monkeypatch.setattr(math_operation, 'add', custom_add)

def test_addition_with_mocked_function(monkeypatched_add):
    result = math_operation.add(3, 4)
    assert result == 12  # 3 * 4 = 12

if __name__ == '__main__':
    # Run the tests using pytest
    pytest.main([__file__])


In pytest, the pytest.main() function is a convenient way to programmatically invoke the pytest testing framework from within a Python script. It allows you to run tests, collect and report results, and customize test execution programmatically.

In Python, \_\_name__ is a special built-in variable that is used to determine whether a Python script is being run as the main program or if it is being imported as a module into another script. The value of \_\_name__ depends on how the Python interpreter is executing the code.

When a Python script is run, the interpreter sets the \_\_name__ variable based on the following rules:

If the script is the main program being executed (not imported as a module), \_\_name__ is set to "\_\_main__".

If the script is being imported as a module into another script, \_\_name__ is set to the name of the module (i.e., the filename without the .py extension).

This distinction allows you to write code that can be both used as a standalone program and imported as a module into other programs without the code being executed unintentionally.

In [1]:
# Code to define a function  
def anything():  
    print('Value of the __name__ : ', __name__)  
      
anything()  

Value of the __name__ :  __main__


In [ ]:
print(f"The value of __file__ is: {__file__}")

# Output: The value of __file__ is: /Users/kristongabor/Desktop/Python training/print_filepath.py

In Python, \_\_file__ is a special variable that provides the path to the script or module that is currently being executed. The variable contains the absolute or relative path to the Python source file.

Here's a brief explanation:

When a Python script or module is being executed, \_\_file__ is set to the path of that script or module.

If the script is being run as the main program, \_\_file__ will contain the absolute or relative path to the script.

If the script is being imported as a module, \_\_file__ will contain the path to the module's source file.

Testing delattr:

In [7]:
# my_module_test.py
import pytest

class A:
        x = 1

def test_delattr():
    monkeypatch = pytest.MonkeyPatch()
    monkeypatch.delattr(A, 'x')
    assert not hasattr(A, 'x')
    monkeypatch.undo()

def test_attr():
    assert A.x == 1

if __name__ == '__main__':
    # Run the tests using pytest
    pytest.main([__file__])

Consider the following scenarios:

1. Modifying the behavior of a function or the property of a class for a test e.g. there is an API call or database connection you will not make for a test but you know what the expected output should be. Use monkeypatch.setattr to patch the function or property with your desired testing behavior. This can include your own functions. Use monkeypatch.delattr to remove the function or property for the test. 

2. Modifying the values of dictionaries e.g. you have a global configuration that you want to modify for certain test cases. Use monkeypatch.setitem to patch the dictionary for the test. monkeypatch.delitem can be used to remove items.

3. Modifying environment variables for a test e.g. to test program behavior if an environment variable is missing, or to set multiple values to a known variable. monkeypatch.setenv and monkeypatch.delenv can be used for these patches.

4. Use monkeypatch.setenv("PATH", value, prepend=os.pathsep) to modify $PATH, and monkeypatch.chdir to change the context of the current working directory during a test.

5. Use monkeypatch.syspath_prepend to modify sys.path which will also call pkg_resources.fixup_namespace_packages and importlib.invalidate_caches().

6. Use monkeypatch.context to apply patches only in a specific scope, which can help control teardown of complex fixtures or patches to the stdlib.

See the monkeypatch blog post for some introduction material and a discussion of its motivation.

In this example, monkeypatch.setattr is used to patch Path.home so that the known testing path Path("/abc") is always used when the test is run. This removes any dependency on the running user for testing purposes. monkeypatch.setattr must be called before the function, which will use the patched function.

In [ ]:
# monkeypatch_2.py

import pytest
from pathlib import Path


def getssh():
    """Simple function to return expanded homedir ssh path."""
    return Path.home() / ".ssh"


def test_getssh(monkeypatch):
    # mocked return function to replace Path.home
    # always return '/abc'
    def mockreturn():
        return Path("/abc")

    # Application of the monkeypatch to replace Path.home
    # with the behavior of mockreturn defined above.
    monkeypatch.setattr(Path, "home", mockreturn)

    # Calling getssh() will use mockreturn in place of Path.home
    # for this test with the monkeypatch.
    x = getssh()
    assert x == Path("/abc/.ssh")

y = getssh()
print(y) # Output: /Users/kristongabor/.ssh

if __name__ == '__main__':
    # Run the tests using pytest
    pytest.main([__file__])